In [14]:
#import config.py that lists out twitter developer api keys to secure a connection between twitter and python app
from config import *
import tweepy as tw
import pandas as pd
import csv
#Importing emoji and nltk for data cleaning process
import emoji
import nltk
import re
nltk.download('words')
words = set(nltk.corpus.words.words())

In [15]:
#file open
file = open('/Users/aj/Downloads/fourth_version_tweeter_dataset.csv')
type(file)
df = pd.read_csv(file)
df.head()
#/Users/aj/Downloads/sephora_website_dataset.csv

,brand,tweet_id,tweet_date,follower_count,retweets,location,tweet_text
0,AERIN,1460799974095015939,2021-11-17 02:40:28+00:00,81,0,"SAN DIEGO, CA","B""@RTCATH95 HE'S AWFFULLY STRANGE AINT HE?"""
1,AERIN,1460799337655590919,2021-11-17 02:37:57+00:00,595,0,NaN,B'@JUST_LIKE_AERIN LYSE IS A SOLID NOT ENTIREL...
2,AERIN,1460798057801392131,2021-11-17 02:32:51+00:00,81,0,"SAN DIEGO, CA","B""@SNOVATORE TRUE LET'S HOMOSEXUALLY TERRORIZE..."
3,AERIN,1460797365879590916,2021-11-17 02:30:06+00:00,1042,0,NaN,B'@SPACESHINJI NGL IF I WERE TO GO WITH SOMETH...
4,AERIN,1460796093441232900,2021-11-17 02:25:03+00:00,595,0,NaN,B'@JUST_LIKE_AERIN ALPHINAUD IS YOUR NEW MIDDL...


In [6]:
#Converting bytes data to string data
df["tweet_text"] = df["tweet_text"].str.decode("utf-8")

In [16]:
#Data cleaning and creating a new column with cleaned textual data
clean_tweets = []
for key, tweet in df["tweet_text"].iteritems():
    tweet = tweet.lower()
    tweet = re.sub(r"b","", tweet)
    tweet = re.sub("@[A-Za-z0-9_]+","", tweet)
    tweet = re.sub("#[A-Za-z0-9_]+","", tweet)
    tweet = re.sub(r"http\S+", "", tweet)
    tweet = re.sub(r"www.\S+", "", tweet)
    tweet = re.sub('[()!?]', ' ', tweet)
    tweet = re.sub('\[.*?\]',' ', tweet)
    tweet = re.sub("[^a-z0-9]"," ", tweet)
    tweet = ''.join(c for c in tweet if c not in emoji.UNICODE_EMOJI) #Remove Emojis
    tweet = tweet.replace("#", "").replace("_", " ") #Remove hashtag sign but keep the text
    tweet = " ".join(w for w in nltk.wordpunct_tokenize(tweet) \
         if w.lower() in words or not w.isalpha())
    clean_tweets.append(tweet)
    
df["clean_tweet"] = clean_tweets


In [17]:
df

,brand,tweet_id,tweet_date,follower_count,retweets,location,tweet_text,clean_tweet
0,AERIN,1460799974095015939,2021-11-17 02:40:28+00:00,81,0,"SAN DIEGO, CA","B""@RTCATH95 HE'S AWFFULLY STRANGE AINT HE?""",he s strange aint he
1,AERIN,1460799337655590919,2021-11-17 02:37:57+00:00,595,0,NaN,B'@JUST_LIKE_AERIN LYSE IS A SOLID NOT ENTIREL...,lyse is a solid not entirely fantasy sounding ...
2,AERIN,1460798057801392131,2021-11-17 02:32:51+00:00,81,0,"SAN DIEGO, CA","B""@SNOVATORE TRUE LET'S HOMOSEXUALLY TERRORIZE...",true let s terrorize it
3,AERIN,1460797365879590916,2021-11-17 02:30:06+00:00,1042,0,NaN,B'@SPACESHINJI NGL IF I WERE TO GO WITH SOMETH...,if i were to go with something related it woul...
4,AERIN,1460796093441232900,2021-11-17 02:25:03+00:00,595,0,NaN,B'@JUST_LIKE_AERIN ALPHINAUD IS YOUR NEW MIDDL...,is your new middle name
...,...,...,...,...,...,...,...,...
20066,STILA,1461068914452930565,2021-11-17 20:29:09+00:00,71,0,"PHILADELPHIA, PA","B""SO GOOD I HAD TO SHARE! CHECK OUT ALL THE IT...",so good i had to share check out all the i m l...
20067,STILA,1461053720687362049,2021-11-17 19:28:46+00:00,450,0,NaN,"B""SO GOOD I HAD TO SHARE! CHECK OUT ALL THE IT...",so good i had to share check out all the i m l...
20068,STILA,1461046024978325508,2021-11-17 18:58:11+00:00,168,0,"RALEIGH, NC","B""CHECK OUT THIS LISTING I JUST ADDED TO MY #P...",check out this listing i just added to my clos...
20069,STILA,1461016962100830210,2021-11-17 17:02:42+00:00,499767,0,NaN,B'EVERYTHING YOU NEED TO KNOW ABOUT THE STAY A...,everything you need to know the stay all day x...


In [20]:
# Import TextBlob for text based sentimental analysis
from textblob import TextBlob

In [21]:
#Perform sentimental analysis using TextBlob
def sentiment_func(tweet):
    try:
        return TextBlob(tweet).sentiment
    except:
        return None

In [22]:
df["sentiment"] = df["clean_tweet"].apply(sentiment_func)
df["sentiment"].head()

0                                 (-0.05, 0.15)
1     (0.10666666666666666, 0.3516666666666667)
2                                  (0.35, 0.65)
3                                    (0.0, 0.4)
4    (0.06818181818181818, 0.22727272727272727)
Name: sentiment, dtype: object

In [24]:
#Divide the sentimental analysis figures into two columns, polarity and subjectivity 
df["sentiment"][0][0]
df['polarity'] = df["sentiment"].apply(lambda x: x[0])
df['subjectivity'] = df["sentiment"].apply(lambda x: x[1])
df[['clean_tweet','sentiment','polarity','subjectivity']].head()

,clean_tweet,sentiment,polarity,subjectivity
0,he s strange aint he,"(-0.05, 0.15)",-0.050000,0.150000
1,lyse is a solid not entirely fantasy sounding ...,"(0.10666666666666666, 0.3516666666666667)",0.106667,0.351667
2,true let s terrorize it,"(0.35, 0.65)",0.350000,0.650000
3,if i were to go with something related it woul...,"(0.0, 0.4)",0.000000,0.400000
4,is your new middle name,"(0.06818181818181818, 0.22727272727272727)",0.068182,0.227273


In [25]:
#Assign words to polarity column
sentiment_word = []
for key, value in df["polarity"].iteritems():
    if value > 0:
         value ='positive'
    elif value == 0:
         value ='neutral'
    else:
         value ='negative'
    sentiment_word.append(value)
df["sentiment_word"] = sentiment_word
df[['clean_tweet','sentiment_word']]

,clean_tweet,sentiment_word
0,he s strange aint he,negative
1,lyse is a solid not entirely fantasy sounding ...,positive
2,true let s terrorize it,positive
3,if i were to go with something related it woul...,neutral
4,is your new middle name,positive
...,...,...
20066,so good i had to share check out all the i m l...,positive
20067,so good i had to share check out all the i m l...,positive
20068,check out this listing i just added to my clos...,neutral
20069,everything you need to know the stay all day x...,neutral


In [29]:
#Calculate the number of positive, negative and neutral tweets 
tweets = df['sentiment_word'].count()
positive = df[df['sentiment_word'] == 'positive'].shape[0]
negative = df[df['sentiment_word'] == 'negative'].shape[0]
neutral = df[df['sentiment_word'] == 'neutral'].shape[0]

In [33]:
#Calculate the percentage of positive, negative and neutral words based on 50 recent searchs and display 5 tweets per each sentiment
pd.set_option('display.max_colwidth', None)
print("Among the recent 50 tweets on search term, 'Armani Beauty'")
print("Positive tweets percentage: {} %".format(100*positive/tweets))
print(df.loc[df['sentiment_word'] == 'positive',['tweet_text']].head())
print("----------------------------------------------------------------")
print("Negative tweets percentage: {} %".format(100*negative/tweets))
print(df.loc[df['sentiment_word'] == 'negative',['tweet_text']].head())
print("----------------------------------------------------------------")
print("Neutral tweets percentage: {} % ".format(100*neutral/tweets))
print(df.loc[df['sentiment_word'] == 'neutral',['tweet_text']].head())

Among the recent 50 tweets on search term, 'Armani Beauty'
Positive tweets percentage: 45.822330725922974 %
                                                                                                                                                                                                                                                  tweet_text
1                                                            B'@JUST_LIKE_AERIN LYSE IS A SOLID NOT ENTIRELY FANTASY SOUNDING MIDDLE NAME. COULD EASILY BE LIKE \XE2\X80\X9CIT\XE2\X80\X99S AN OLD BRITISH NAME\XE2\X80\X9D OR SOMETHING ALONG THOSE LINES.'
2                                                                                                                                                                                                         B"@SNOVATORE TRUE LET'S HOMOSEXUALLY TERRORIZE IT"
4                                                                                                                                    

In [192]:
df.to_csv(r'/Users/aj/Downloads/tweet_dataset_with_sentiments.csv', index = False)